In [1]:
from mistralai import Mistral
from dotenv import load_dotenv
import datauri
import os
import logging
import pandas as pd

In [2]:
fecha_hoy = pd.Timestamp.now().strftime("%Y-%m-%d")
file_name_log = rf"C:\Users\davila\Documents\Python Projects\Lector PDF Mistral\Log\lectura_pdf {fecha_hoy}.log" ## cambiar la ruta 
# Configurar logging para escribir en un archivo
logging.basicConfig(
    filename= file_name_log,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    encoding="utf-8"
)


In [3]:
# Cargar las variables del archivo .env
load_dotenv()

True

In [4]:
api_key_mistral = os.getenv('MISTRAL_API_KEY')  # Reemplaza con tu clave API

In [5]:
#Ejecutar el cliente de Mistral
client = Mistral(api_key=api_key_mistral)

In [6]:
def upload_pdf(filename):
  """
  Funcion para subir un archivo PDF a Mistral y obtener una URL firmada.
  """
  uploaded_pdf = client.files.upload(
    file={
      "file_name": filename,
      "content": open(filename, "rb"),
    },
    purpose="ocr"
  )
  logging.info(f"Archivo {filename} subido correctamente.")
  # Obtener la URL firmada
  signed_url = client.files.get_signed_url(file_id=uploaded_pdf.id)
  return signed_url.url


In [7]:
def check_pdf(filename):
    """
    Funcion para verificar si un archivo PDF ya fue subido a Mistral.
    """
    try:
    # Verificar si el archivo ya fue subido
        list_pdf = client.files.list()
        list_filename_id_pdf = [ [file.filename,file.id]  for file in list_pdf.data]
        list_filename_pdf = [file.filename for file in list_pdf.data]
        
        if filename in list_filename_pdf:
            # encontrar el id del archivo
            id_pdf = [file[1] for file in list_filename_id_pdf if file[0] == filename][0]
            # Obtener la URL firmada
            signed_url = client.files.get_signed_url(file_id=id_pdf)
            logging.info(f"El archivo {filename} ya fue subido a Mistral.")
            print(f"El archivo {filename} ya fue subido a Mistral.")
            return signed_url.url
        else:
            logging.info(f"El archivo {filename} no fue subido a Mistral.")
            print(f"El archivo {filename} no está subido a Mistral.")
            return False
    except Exception as e:
        logging.error(f"Error al verificar el archivo {filename}: {e}")
        print(f"Error al verificar el archivo {filename}: {e}")
        return None

In [8]:
def create_markdown_file(ocr_response, output_filename):
  """
  Función para crear un archivo markdown a partir de la respuesta de OCR.
  """
  with open(output_filename, "wt",encoding="utf-8") as f:
    for page in ocr_response.pages:
      f.write(page.markdown)
  logging.info(f"Se ha generado el archivo markdown {output_filename} con éxito.")
  print(f"Se ha generado el archivo markdown {output_filename} con éxito.")

In [9]:
def get_ocr_response(signed_url):
  """
  Función para obtener la respuesta de OCR a partir de la URL firmada.
  """
  try:
    ocr_response = client.ocr.process(model="mistral-ocr-latest",
                       document={
                          "type": "document_url",
                          "document_url": signed_url,
                        },
                        include_image_base64=True
                      )
    return ocr_response
  except Exception as e:
    logging.error(f"Error al procesar el OCR: {e}")
    print(f"Error al procesar el OCR: {e}")
    return None

In [10]:
input_file = r'C:\Users\davila\Documents\Python Projects\Lector PDF Mistral\Inputs\MC0003272.pdf'
name_input_file = os.path.basename(input_file)
name_input_file_without_ext = os.path.splitext(name_input_file)[0]
output_file = fr'C:\Users\davila\Documents\Python Projects\Lector PDF Mistral\Outputs\Markdowns\{name_input_file_without_ext}.md'

In [11]:
# Verificar si el archivo ya fue subido
chequeo = check_pdf(name_input_file)

El archivo MC0003272.pdf ya fue subido a Mistral.


In [12]:
if chequeo == False:
    # Subir el archivo a Mistral
    url = upload_pdf(input_file)
    logging.info(f"El archivo {name_input_file} fue subido a Mistral.")
    print(f"El archivo {name_input_file} fue subido a Mistral.")
    # Realizar OCR en el archivo PDF   
else:
    logging.info(f"El archivo {name_input_file} ya fue subido a Mistral.") 
    url = upload_pdf(input_file)

In [13]:
texto_plano = get_ocr_response(url)

In [14]:
# convertir response a markdown
create_markdown_file(texto_plano, output_file)

Se ha generado el archivo markdown C:\Users\davila\Documents\Python Projects\Lector PDF Mistral\Outputs\Markdowns\MC0003272.md con éxito.
